In [1]:
from mountaintools import client as mt

Loading environment from: /home/magland/.mountaintools/.env


In [2]:
mt.configRemoteReadonly(collection='spikeforest', share_id='69432e9201d0')

CAIRIO CONFIG: remote database spikeforest (readonly); remote kb-share 69432e9201d0 (readonly)


In [3]:
X=mt.loadObject(
    key=dict(
        name='spikeforest_results'
    ),
    subkey='visapy_mea'
)

In [5]:
X.keys()

dict_keys(['studies', 'recordings', 'sorting_results', 'aggregated_sorting_results'])

In [40]:
for sr in X['sorting_results']:
    study=sr['recording']['study']
    recording=sr['recording']['name']
    sorter=sr['sorter']
    true_units_info=mt.loadObject(path=sr['recording']['summary']['true_units_info'])
    comparison_with_truth=mt.loadObject(path=sr['comparison_with_truth']['json'])
    units_by_id=dict()
    for unit in true_units_info:
        unit_id=unit['unit_id']
        units_by_id[unit_id]=unit
    for ii,unit in comparison_with_truth.items():
        unit_id=unit['unit_id']
        for key in unit:
            units_by_id[unit_id][key]=unit[key]
    for id0,unit in units_by_id.items():
        unit['study']=study
        unit['recording']=recording
        unit['sorter']=sorter['name']
        print(unit)
        raise Exception('test')
    print(units_by_id.values())
    #print(comparison_with_truth)

{'unit_id': 1, 'snr': 0.654514033190837, 'peak_channel': 10, 'num_events': 1031, 'firing_rate': 3.4366666666666665, 'accuracy': '0.9461', 'best_unit': 28, 'matched_unit': 28, 'num_matches': 982, 'num_false_negatives': 49, 'num_false_positives': 7, 'f_n': '0.0475', 'f_p': '0.0071', 'study': 'visapy_mea', 'recording': 'set1', 'sorter': 'MountainSort4-thr3'}


Exception: test

In [25]:
X['sorting_results']

[{'recording': {'name': 'set1',
   'study': 'visapy_mea',
   'directory': 'kbucket://15734439d8cf/groundtruth/visapy_mea/set1',
   'description': 'One of the recordings in the visapy_mea study',
   'summary': {'computed_info': {'samplerate': 32000.0,
     'num_channels': 30,
     'duration_sec': 300.0},
    'plots': {},
    'true_units_info': 'sha1://8c5387cdd776d85e293f2c1c8032f0142ae81bae/true_units_info.json'}},
  'sorter': {'name': 'MountainSort4-thr3',
   'processor_name': 'MountainSort4',
   'params': {'detect_sign': -1,
    'adjacency_radius': 50,
    'detect_threshold': 3}},
  'firings_true': 'kbucket://15734439d8cf/groundtruth/visapy_mea/set1/firings_true.mda',
  'processor_name': 'MountainSort4',
  'processor_version': '4.2.0',
  'execution_stats': {'start_time': 1551979549.3653011,
   'end_time': 1551979664.5984027,
   'elapsed_sec': 115.23310160636902},
  'console_out': 'sha1://253fe5fb39c1bab5f2119820c303ac08f6191cad/console_out.txt',
  'container': 'default',
  'firings':

In [7]:
Y=mt.loadObject(path=X['aggregated_sorting_results'])

In [15]:
Y['recording_sorting_results'][0]['num_false_positives']

[7, 49, 82, 12, 391, 5, 11, 10, 258, 13, 1, 15, 28, 12, 33, 38]

In [16]:
X['studies']

[{'name': 'visapy_mea',
  'study_set': 'visapy_mea',
  'directory': 'kbucket://15734439d8cf/groundtruth/visapy_mea',
  'description': ''}]

In [17]:
X['recordings']

[{'name': 'set1',
  'study': 'visapy_mea',
  'directory': 'kbucket://15734439d8cf/groundtruth/visapy_mea/set1',
  'description': 'One of the recordings in the visapy_mea study',
  'summary': {'computed_info': {'samplerate': 32000.0,
    'num_channels': 30,
    'duration_sec': 300.0},
   'plots': {},
   'true_units_info': 'sha1://8c5387cdd776d85e293f2c1c8032f0142ae81bae/true_units_info.json'}},
 {'name': 'set2',
  'study': 'visapy_mea',
  'directory': 'kbucket://15734439d8cf/groundtruth/visapy_mea/set2',
  'description': 'One of the recordings in the visapy_mea study',
  'summary': {'computed_info': {'samplerate': 32000.0,
    'num_channels': 30,
    'duration_sec': 300.0},
   'plots': {},
   'true_units_info': 'sha1://c048908a3a557bfb9741b7378e7fdebfd8f609e3/true_units_info.json'}},
 {'name': 'set3',
  'study': 'visapy_mea',
  'directory': 'kbucket://15734439d8cf/groundtruth/visapy_mea/set3',
  'description': 'One of the recordings in the visapy_mea study',
  'summary': {'computed_info